In [1]:
import json
from collections import defaultdict
from sklearn import linear_model, model_selection
import numpy as np
import random
import gzip
import dateutil.parser
import math
import mido
import pretty_midi
import joblib
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec
import collections
import os
import tarfile
import tables
import hdf5_getters as GETTERS
from midi_methods import *

%store -r

In [2]:
"""
f = tarfile.open("resources/lmd_matched.tar.gz")
f.extractall()
f.close

f = tarfile.open("resources/lmd_matched_h5.tar.gz")
f.extractall()
f.close
"""



'\nf = tarfile.open("resources/lmd_matched.tar.gz")\nf.extractall()\nf.close\n\nf = tarfile.open("resources/lmd_matched_h5.tar.gz")\nf.extractall()\nf.close\n'

In [3]:
# https://github.com/craffel/pretty-midi/blob/main/Tutorial.ipynb
test_song_path = 'lmd_matched/A/A/A/TRAAAGR128F425B14B/1d9d16a9da90c090809c153754823c2b.mid'

song_pm = pretty_midi.PrettyMIDI(test_song_path)
all_instruments = song_pm.instruments
# Instrument.program to get program number
all_instruments


[Instrument(program=62, is_drum=False, name="main synth          "),
 Instrument(program=62, is_drum=False, name="echo synth          "),
 Instrument(program=48, is_drum=False, name="main strings h      "),
 Instrument(program=48, is_drum=False, name="main strings m      "),
 Instrument(program=48, is_drum=False, name="main strings l      "),
 Instrument(program=33, is_drum=False, name="main bass           "),
 Instrument(program=0, is_drum=True, name="drums + fills       "),
 Instrument(program=14, is_drum=False, name="xtras               "),
 Instrument(program=6, is_drum=False, name="harpsichord         "),
 Instrument(program=19, is_drum=False, name="verse vibe h        "),
 Instrument(program=19, is_drum=False, name="verse vibe m        "),
 Instrument(program=19, is_drum=False, name="verse vibe l        "),
 Instrument(program=1, is_drum=False, name="verse piano         "),
 Instrument(program=64, is_drum=False, name="verse xtra          "),
 Instrument(program=48, is_drum=False,

In [4]:
main_synth = all_instruments[0]
main_synth

Instrument(program=62, is_drum=False, name="main synth          ")

In [5]:
notes_main_synth = main_synth.notes
notes_main_synth[0], notes_main_synth[-1] 

(Note(start=0.018229, end=0.184896, pitch=69, velocity=100),
 Note(start=215.851562, end=215.989583, pitch=71, velocity=100))

In [6]:
test_hdf5_path = 'lmd_matched_h5\A\A\A\TRAAAGR128F425B14B.h5'
h5 = GETTERS.open_h5_file_read(test_hdf5_path)
h5.root.metadata

/metadata (Group) 'metadata about the song'
  children := ['artist_terms' (EArray), 'artist_terms_freq' (EArray), 'artist_terms_weight' (EArray), 'similar_artists' (EArray), 'songs' (Table)]

In [7]:
h5.root.metadata.songs.cols.genre

/metadata/songs.cols.genre (Column(1,), string, idx=None)

In [8]:
#https://github.com/tbertinmahieux/MSongsDB/blob/master/PythonSrc/hdf5_getters.py